In [ ]:
import requests
import json
import seaborn as sns
import pandas as pd
pd.set_option('display.max_columns',100)
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

In [ ]:
# import the csv file that has all of our collected data from the two NHL APIs
df = pd.read_csv('Final_NHL_stats.csv')

In [ ]:
# set the index to our unique team/season IDs
df.set_index('teamID_seasonID',inplace=True)

In [ ]:
# drop columns that are repeated from the two API calls (regular season and defense stats)
df.drop(columns=['Unnamed: 0','wins_y','shootingPctg_y','ot','losses_y','faceoffsLost','faceoffsWon',
                 'faceoffWinPctg','faceoffs','gamesPlayed_y'],axis=1, inplace = True)

In [ ]:
# create a new column to reflect a team's win percentage for the season
# this will be our target variable for our OLS model
df['win_percentage']=df['wins_x']/df['gamesPlayed_x']

In [ ]:
#histogram and normal probability plot of our target variable to see distribution
sns.distplot(df['win_percentage'], fit=stats.norm, bins=20, kde=False);
fig = plt.figure()
res = stats.probplot(df['win_percentage'], plot=plt)

In [ ]:
# create a list of independent variables, removing variables which are strings
features = features[1:-4]

In [ ]:
#break the colums into groups to plot 4 on a row at a time
n = 4
row_groups= [features[i:i+n] for i in range(0, len(features), n) ]

In [ ]:
# create scatter plots for every independent variable vs. our target variable
for i in row_groups:
    pp = sns.pairplot(data=df, y_vars=['win_percentage'],x_vars=i, kind="reg", height=3)

pp.savefig('Pairplot.png')

In [ ]:
# investigate distribution of specific variable which seems to have outliers
sns_plot = sns.distplot(df['faceOffsWon'], axlabel = "Number of Faceoffs Won by Team",bins=30,color='g').set_title("Faceoff Distribution")
fig = sns_plot.get_figure()
fig.savefig('FaceOff_dist.png')

In [ ]:
# find the rows associated with the identified outliers 
df[(df['faceOffsWon']>1200) & (df['faceOffsWon']<1600)]

In [ ]:
# remove the rows shown above which refer to a season which had a partial lockout
# as these values do not reflect a standard regular season in the NHL

ixl = list(df.index)
lockout = [i for i in ixl if '20122013' in i]
df.drop(index=lockout,axis=0,inplace=True)

In [ ]:
# recreate distribution for faceoffs won after removing outliers
sns_plot = sns.distplot(df['faceOffsWon'], axlabel = "Number of Faceoffs Won by Team",bins=30,color='g').set_title("Faceoff Distribution")
fig = sns_plot.get_figure()
fig.savefig('FaceOff_dist_no_outliers.png')

In [ ]:
# create individual plots for certain variables after removing outliers
plot = sns.pairplot(data=df, y_vars=['win_percentage'],
             x_vars='shotsFor', kind="reg", 
             height=4,aspect=1.5)
plot.fig.suptitle('Correlation between total shot attempts and win percentage')
# plot.axes.set_xlabel('Number of faceoffs lost in a season')
plt.xlabel("Number of shot attempts in a season")
plt.ylabel('Win percentage')

plot.savefig('Corr1.png')

In [ ]:
plot = sns.pairplot(data=df, y_vars=['hits'],
             x_vars='shotsFor', kind="reg", 
             height=4,aspect=1.5)
plot.fig.suptitle('Correlation between total hits attempts and win percentage')
# plot.axes.set_xlabel('Number of faceoffs lost in a season')
plt.xlabel("Number of hits in a season")
plt.ylabel('Win percentage')

plot.savefig('Corr2.png')

In [ ]:
plot = sns.pairplot(data=df, y_vars=['savePctg'],
             x_vars='shotsFor', kind="reg", 
             height=4,aspect=1.5)
plot.fig.suptitle('Correlation between save percentage and win percentage')
# plot.axes.set_xlabel('Number of faceoffs lost in a season')
plt.xlabel("Goalie's save percentage")
plt.ylabel('Win percentage')

plot.savefig('Corr4.png')

In [ ]:
# check to make sure there are no missing values in the dataframe
null_values = sns.heatmap(df.isnull(), cbar=False)
fig = null_values.get_figure()
fig.savefig('Null_values.png')

In [ ]:
# create a column for a categorical variable, adding a 1
# for the original 6 teams in the NHL, otherwise add a 0
df['Original_6']=np.where(((df['teamFullName']=='Boston Bruins') | (df['teamFullName']=='Chicago Blackhawks') |
                    (df['teamFullName']=='Detroit Red Wings') | (df['teamFullName']=='New York Rangers')
                    | (df['teamFullName']=='Toronto Maple Leafs') | (df['teamFullName']=='Montreal Canadiens')),1,0)

In [ ]:
# create an interaction variable that aims to measure
# defensive strength of a team
df['defensive_strength'] = df['hits']*df['takeaways']*df['penaltyKillPercentage']*df['shotsAllowed']

In [ ]:
# create a new interaction variable which is a proxy for puck possession
df['puck_possession']=(df['blockedShots']*df['shotsFor']*df['missedShots'])

In [ ]:
def CorrMtx(df, dropDuplicates = True):

    # Your dataset is already a correlation matrix.
    # If you have a dateset where you need to include the calculation
    # of a correlation matrix, just uncomment the line below:
    # df = df.corr()

    # Exclude duplicate correlations by masking uper right values
    if dropDuplicates:    
        mask = np.zeros_like(df, dtype=np.bool)
        mask[np.triu_indices_from(mask)] = True

    # Set background color / chart style
    sns.set_style(style = 'white')

    # Set up  matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # Add diverging colormap from red to blue
    cmap = sns.diverging_palette(250, 10, as_cmap=True)

    # Draw correlation plot with or without duplicates
    if dropDuplicates:
        sns.heatmap(df, mask=mask, cmap=cmap, 
                square=True,
                linewidth=.5, cbar_kws={"shrink": .5}, ax=ax)
    else:
        sns.heatmap(df, cmap=cmap, 
                square=True,
                linewidth=.5, cbar_kws={"shrink": .5}, ax=ax)

In [ ]:
# run a correlation on all of our variables
# print the heat map 
corr = df.corr()
CorrMtx(corr, dropDuplicates = True)

In [ ]:
# isolate the independent variables that we want to run through our model
features = ['faceOffWinPercentage',
       'faceOffsWon', 'goalsAgainstPerGame', 'goalsPerGame',
       'powerPlayPercentage', 'savePctg', 'shootingPctg_x',
       'shotsAllowed', 'blockedShots', 'giveaways', 'goalsFor', 
        'missedShots', 'shotsFor']

In [ ]:
# run our first test model using most of our features 
slr_model_1 = sm.OLS(endog=df['win_percentage'], exog=sm.add_constant(df[features])).fit()

slr_model_1.summary()

In [ ]:
# function that shows the distribution of the residual values from a model
# and creates a scattor plot of the residuals vs. the target variable
def checkresiduals(df, target, sm_model):
    # checking for our model - Homoscedasticity,  Independence of residuals
    pred_val = sm_model.fittedvalues.copy()
    true_val = df[target].values.copy()
    residual = true_val - pred_val
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
    ax1.hist(residual, density=True, bins=30)
    ax2.scatter(df[target],residual)
    ax2.set_title('Residual Scatterplot')
    plt.show()

In [ ]:
# check the residuals of the first model
checkresiduals(df, 'win_percentage', slr_model_1)

In [ ]:
# fine-tune our independent variables by removing variables 
# with large p-values (indicating they do not have statistical significance)
features = ['faceOffsLost', 'powerPlayOpportunities',
            'savePctg', 'puck_possession']

In [ ]:
df[features].head()

In [ ]:
# run a second test model
# this model has a lower R-squared but now our variables
# are all statistically significant
slr_model_2 = sm.OLS(endog=df['win_percentage'], exog=sm.add_constant(df[features])).fit()

In [ ]:
slr_model_2.summary()

In [ ]:
# scale the features so that our coefficients are on the same scale
# and can be interpreted 
scaler = StandardScaler()
scaler.fit(df[features])
scaled_features = scaler.transform(df[features])

scaled_features_df = pd.DataFrame(scaled_features, columns=features, index=df.index)
scaled_features_df.head()

In [ ]:
# run a new model with same features from model 2 
# but now the features have been scaled
# R-squared should be the same as it was in model 2
scaled_features_model_2= sm.OLS(endog=df['win_percentage'], exog=sm.add_constant(scaled_features_df)).fit()

scaled_features_model_2.summary()

In [ ]:
# we know that our features above returned an R-squared of .4
# so let's try to do better
# add in two more features, one for average goals against per game 
# and average goals for per game
features = ['faceOffsLost', 'goalsAgainstPerGame','goalsPerGame',
            'giveaways','savePctg', 'puck_possession']

In [ ]:
# scale the features prior to running the next model
scaler = StandardScaler()
scaler.fit(df[features])
scaled_features = scaler.transform(df[features])

scaled_features_df = pd.DataFrame(scaled_features, columns=features, index=df.index)
scaled_features_df.head()

In [ ]:
# run the third model
# the goal data has now increased the R-squared
# and all of our p-values are significant
slr_model_3 = sm.OLS(endog=df['win_percentage'], exog=sm.add_constant(scaled_features_df)).fit()

slr_model_3.summary()

In [ ]:
# check the residuals of our new third model
checkresiduals(df, 'win_percentage', slr_model_3)

In [ ]:
# we noticed additional outliers in the data
# the NHL attempted to implement stronger rules against obstruction
# so a variety of the data from the 2002 season is lower than the 
# remainder of the data

# remove that season to check if it impacts our results
ixl = list(df.index)
lockout = [i for i in ixl if '20022003' in i]
df_exclude2002 = df.drop(index=lockout,axis=0)

In [ ]:
df_exclude2002.head()

In [ ]:
# scale the features again with our smaller data set
scaler = StandardScaler()
scaler.fit(df_exclude2002[features])
scaled_features = scaler.transform(df_exclude2002[features])

scaled_features_df = pd.DataFrame(scaled_features, columns=features, index=df_exclude2002.index)
scaled_features_df.head()

In [ ]:
# run a model excluding the 2002 season
scaled_features_model_exclude2002 = sm.OLS(endog=df['win_percentage'], exog=sm.add_constant(scaled_features_df)).fit()

scaled_features_model_exclude2002.summary()

In [ ]:
checkresiduals(df_exclude2002, 'win_percentage', scaled_features_model_exclude2002)

In [ ]:
# the model immaterially impacted by removing the 2002 seaon
# so we chose to keep that season's data in our final models (slr_model_2 and slr_model_3)